# Chatbot infos corona

Dans ce cas pratique on va essayer d'entraîner un chatbot qui permette de répondre automatiquement aux questions de l'utilisateur sur le coronavirus à partir de multiples informations que l'on a regroupées dans le fichier `infos_corona.txt`. L'idée est que le chatbot renvoie la ou les phrases utilisant le plus de termes similaires à ceux utilisés dans la question de l'utilisateur.   
La démarche est similaire à celle présentée dans le notebook `chatbot_wiki`.  

- En utilisant la fonction `open` (et en précisant le paramètre `encoding` adéquat!), importez le texte dans une chaîne de caractères que vous nommerez `texte`.  
- Procédez à quelques nettoyages : 
    - passez le texte en minuscules.  
    - un acronyme (n.c.a.) va fausser notre tokenization en phrases, remplacez-le par autre chose.  
    - le virus est appelé covid-19 ou coronavirus, faites en sorte qu'un terme unique soit utilisé.   
- Avec la fonction `nltk.sent_tokenize`, passez votre chaîne de caractères `texte` en une liste de phrases que vous appelerez `phrases_token`.  
- Il y a beaucoup de questions dans cette liste, or on veut des réponses. Supprimez-les (_une méthode_ : boucle sur la liste et suppression avec `del` des éléments contenant un "?". __Attention__ : il faut faire partir la boucle de l'indice le plus grand vers 0, sinon le fait de supprimer des éléments fausse l'indexation!).     
- Récupérez un vecteur de stop words français avec la fonction `get_stop_words` du module `stop_words` 

On a maintenant tout ce dont on a besoin pour faire notre matrice TF-IDF! Vous pouvez déjà la fitter sur vos infos :  
- Stockez le résultat de `TfidfVectorizer` en fixant le paramètre `stop_words` avec les stop words français.  
- Fittez la fonction sur votre liste de phrases `phrases_token` et stockez ce résultat dans un objet `tf_idf_chat`.  

Il faut maintenant définir une fonction que vous appelerez dans votre chatbot. Celle-ci doit :  
- Prendre en entrée la phrase entrée par l'utilisateur et la mettre dans une liste.    
- Créer la matrice TF-IDF des infos avec `tf_idf_chat.transform()`. 
- Créer la matrice TF-IDF pour la phrase de l'utilisateur avec `tf_idf_chat.transform()`.  
- Calculez la similarité entre la phrase de l'utilisateur et le reste des phrases avec `sklearn.metrics.pairwise.cosine_similarity`.  
- Renvoyer en réponse la phrase avec la similarité la plus grande, ou un message d'erreur s'il n'y a pas de similarité. _Alternative un peu plus complexe_ : Vous pouvez aussi renvoyer plusieurs phrases si plusieurs sont similaires, en les concaténant dans une même chaîne de caractère avec '\n'.join().  

Maintenant il n'y a plus qu'à intégrer cette fonction dans votre chatbot et le tester! N'oubliez pas de lui faire dire bonjour et de laisser la possibilité à l'utilisateur de quitter! Bon courage! 


In [1]:
# Imports
import nltk
import numpy as np
import random
import string # to process standard python strings
import re

In [2]:
# On stocke le texte dans une variable
f=open('infos_corona.txt','r',errors = 'ignore',encoding="utf-8-sig")
texte = f.read()

In [3]:
# Phase de nettoyage
texte = texte.lower()
texte = texte.replace("n.c.a.","NCA")
texte = texte.replace("\n"," ")
texte = texte.replace("covid-19","coronavirus")

In [14]:
nltk.download('punkt')
nltk.download('wordnet')
phrase_token = nltk.sent_tokenize(texte) 

[nltk_data] Downloading package punkt to /home/jean/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jean/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
phrase_token


['depuis janvier 2020, une épidémie de coronavirus coronavirus (ex 2019-ncov) s’est propagée depuis la chine.',
 'retrouvez sur cette plateforme toutes les réponses officielles aux questions que vous vous posez sur ce qu’est le coronavirus coronavirus et les recommandations pour votre santé et votre vie quotidienne.',
 'face à l’épidémie , il existe des gestes simples pour préserver votre santé et celle de votre entourage :  se laver les mains très régulièrement tousser ou éternuer dans son coude ou dans un mouchoir  saluer sans se serrer la main, arrêter les embrassades utiliser des mouchoirs à usage unique et les jeter eviter les rassemblements, limiter les déplacements et les contacts.',
 'qu’est-ce que le coronavirus coronavirus ?',
 'les coronavirus sont une grande famille de virus, qui provoquent des maladies allant d’un simple rhume (certains virus saisonniers sont des coronavirus) à des pathologies plus sévères comme le mers-cov ou le sras.',
 'le virus identifié en janvier 202

In [6]:
phrases = []
for phrase in phrase_token:
    if ('?' not in phrase):
        phrases.append(phrase)
print("Nombre des phrase_token:", len(phrase_token), "Nombre de phrases (sans questions):",len(phrases))

Nombre des phrase_token: 557 Nombre de phrases (sans questions): 452


In [8]:
from stop_words import get_stop_words
stop_words = get_stop_words('fr')

In [9]:
# Entrainement du modèle
from sklearn.feature_extraction.text import TfidfVectorizer

TfidfVec = TfidfVectorizer(stop_words = stop_words)
tf_idf_chat = TfidfVec.fit(phrases)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import FrenchStemmer

flag=True
print("Bonjour, je suis COROBOT, et je peux répondre à vos questions à propos du covid-19\nPosez donc votre question:")
while(flag==True):
    question = []
    user_response = input()
    user_response=user_response.lower()
    question.append(user_response)
    tfidf_a = tf_idf_chat.transform(phrases)
    tfidf_q = tf_idf_chat.transform(question)
    vals = cosine_similarity(tfidf_q, tfidf_a)  
    idx=vals.argsort()[0][-1]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-1]
    
    if(user_response!='au revoir'):
        print(f"COROBOT : {phrases[idx]}")
        if(user_response=='merci'):
            flag=False
            print("COROBOT: Je vous en prie")

Bonjour, je suis COROBOT, et je peux répondre à vos questions à propos du covid-19
Posez donc votre question:
dangereux ?
COROBOT : les animaleries restent également ouvertes.
symptomes ?
COROBOT : se déplacer pour donner son sang est autorisé.
symptômes ?
COROBOT : le port du masque chirurgical n’est pas recommandé sans présence de symptômes.
symptomes ?
COROBOT : se déplacer pour donner son sang est autorisé.


KeyboardInterrupt: 

## Rendez votre chatbot encore plus intelligent!  
Donnez la possibilité à l'utilisateur de nourrir le chatbot de nouvelles informations. Vous pouvez par exemple déterminer qu'après avoir tapé "infos", l'utilisateur va rentrer une phrase que vous devrez ajouter aux possibilités de réponses de votre chatbot.  

## Bonus avec stemmatizer : 

On définit maintenant une fonction à indiquer dans le paramètre `tokenizer` de la fonction `TfidfVectorizer`. Cette fonction doit récupérer la racine des mots plutôt que les mots en entier pour trouver des correspondances entre des mots de la même racine même s'ils ne sont pas écrits sous la même forme.  
- Utilisez la fonction `FrenchStemmer` de `nltk.stem.snowball` pour définir :  
    - une première fonction qui renvoie la liste des mots stemmatisés quand on rentre une liste de mots.  
    - Une fonction qui applique cette première fonction à une phrase qu'on tokenize en mots (avec `nltk.word_tokenize`)  
Vous pouvez tester cette fonction sur des phrases pour voir si elle fait bien ce que vous désirez.  
- Ajoutez l'appel à cette fonction dans `TfidfVectorizer`.  


In [17]:
# Test FrenchStemmer
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()

# fonction qui transforme une phrase en liste de mots:
def sentence_to_words(text):
    return nltk.word_tokenize(text) 

# fonction qui "stemmatize"
def stem(sentence):
    stem = ''
    tokenized = sentence_to_words(sentence.lower())
    for word in tokenized:
        stem += ' ' + stemmer.stem(word)
    return stem

print(stem("Les mutations du virus sont incroyables"))

 le mutat du virus sont incroi


In [12]:
# Entrainement du modèle
from sklearn.feature_extraction.text import TfidfVectorizer

TfidfVec = TfidfVectorizer(tokenizer=stem, stop_words = stop_words)
tf_idf_chat = TfidfVec.fit(phrases)

/home/jean/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/home/jean/.local/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens [' ', 'b', 'c', 'e', 'f', 'h', 'i', 'o', 'p', 'q', 'r', 's', 'u', 'v', 'x', 'z', 'ç', 'é', 'ê', 'ù', 'û'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [13]:
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import FrenchStemmer

flag=False
print("Bonjour, je suis COROBOT, et je peux répondre à vos questions à propos du covid-19\nPosez donc votre question:")
while(flag==True):
    question = []
    user_response = input()
    user_response=user_response.lower()
    user_response = stem(user_response)
    question.append(user_response)
    tfidf_a = tf_idf_chat.transform(phrases)
    tfidf_q = tf_idf_chat.transform(question)
    vals = cosine_similarity(tfidf_q, tfidf_a)  
    idx=vals.argsort()[0][-1]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-1]
    
    if(user_response!='au revoir'):
        print(f"COROBOT : {phrases[idx]}")
        if(user_response=='merci'):
            flag=False
            print("COROBOT: Je vous en prie")

Bonjour, je suis COROBOT, et je peux répondre à vos questions à propos du covid-19
Posez donc votre question:
